In [1]:
import pickle
import os
import networkx as nx
import numpy as np
import time
import h5py
import pandas as pd
from itertools import product

In [2]:
class TrafficGridAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.traffic_grid = None
        self.zero_indices = None
        self.zero_count = 0

    def load_data(self):
        with h5py.File(self.file_path, 'r') as f:
            for key in f.keys():  # 读取第一个 key 对应的数据
                data = np.array(f[key])  # 转换为 NumPy 数组
                self.traffic_grid = data[0]  # 取第一个通道
                break  # 只读取第一个 key

    def analyze_traffic(self, threshold=30):
        self.zero_indices = np.argwhere(self.traffic_grid < threshold)
        self.zero_count = self.zero_indices.shape[0]
        return self.zero_count, self.zero_indices.tolist()

    def visualize(self):
        # 绘制原始交通网格灰度图
        plt.figure(figsize=(10, 8))
        plt.imshow(self.traffic_grid, cmap='gray')
        plt.title('Original Traffic Network Visualization')
        plt.axis('off')
        plt.show()
        
        # 创建新灰度图矩阵，初始化为255
        visual_grid = np.full_like(self.traffic_grid, 255)
        
        # 将 zero_indices 对应的节点灰度值设置为0
        for row, col in self.zero_indices:
            visual_grid[row, col] = 0
        
        # 绘制提取的交通网络
        plt.figure(figsize=(10, 8))
        plt.imshow(visual_grid, cmap='gray')
        plt.title('Extracted Traffic Network Visualization')
        plt.axis('off')
        plt.show()

In [3]:
class TrafficDataAnalyzer:
    def __init__(self, march_half_filepath, march_filepath):
        self.march_half_filepath = march_half_filepath
        self.march_filepath = march_filepath
        self.Antwerp_traffic_last_half_march = None
        self.Antwerp_traffic_march = None
        self.average_traffic = None
        self.filtered_indices = None
        self.zero_indices = None

    def load_variable(self, filepath):
        """
        从文件中加载变量。
        """
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
        return data

    def readdata(self, filepath):
        """
        读取数据文件并转换为 NumPy 数组。
        """
        data = self.load_variable(filepath)
        data = np.array(data)
        print(f"Loaded data shape: {data.shape}")
        return data

    def analyze_traffic(self, zero_indices):
        # 读取数据
        self.Antwerp_traffic_last_half_march = self.readdata(self.march_half_filepath)
        self.Antwerp_traffic_march = self.readdata(self.march_filepath)
        
        self.zero_indices = zero_indices
        
        # 计算沿时间轴的平均交通流量
        self.average_traffic = np.mean(self.Antwerp_traffic_march, axis=0)
        
        # 排除 zero_indices 中的无连接节点
        for row, col in self.zero_indices:
            self.average_traffic[row, col] = np.nan

        # 计算 30% 和 95% 分位数
        valid_values = self.average_traffic[~np.isnan(self.average_traffic)]
        lower_bound = np.percentile(valid_values, 30)
        upper_bound = np.percentile(valid_values, 100)
        
        # 筛选出不在 30%-95% 范围内的节点
        valid_mask = ~np.isnan(self.average_traffic)
        outside_range_mask = (self.average_traffic < lower_bound) | (self.average_traffic > upper_bound)
        filtered_mask = valid_mask & outside_range_mask
        
        # 找到不在范围内的节点索引
        self.filtered_indices = np.argwhere(filtered_mask)
        filtered_count = self.filtered_indices.shape[0]
        
        # 创建 DataFrame 并打印结果
        filtered_df = pd.DataFrame(self.filtered_indices, columns=['Row', 'Column'])
        print(f"Filtered Traffic Indices (Outside {lower_bound}-{upper_bound} Range):")
        print(filtered_df)
        print(f"Total number of filtered traffic nodes: {filtered_count}")
        print(f"30% Percentile (Lower Bound): {lower_bound}")
        print(f"95% Percentile (Upper Bound): {upper_bound}")
        
        # 计算描述性统计信息
        for row, col in self.filtered_indices:
            self.average_traffic[row, col] = np.nan
        
        valid_mask_total = ~np.isnan(self.average_traffic)
        remaining_values = self.average_traffic[valid_mask_total].flatten()
        
        q1 = np.percentile(remaining_values, 25)
        median = np.percentile(remaining_values, 50)
        q3 = np.percentile(remaining_values, 75)
        iqr = q3 - q1
        mean = np.mean(remaining_values)
        min_val = np.min(remaining_values)
        max_val = np.max(remaining_values)
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        print(f"Minimum Value: {min_val}")
        print(f"Q1 (25th Percentile): {q1}")
        print(f"Median (50th Percentile): {median}")
        print(f"Q3 (75th Percentile): {q3}")
        print(f"Maximum Value: {max_val}")
        print(f"Mean: {mean}")
        print(f"Lower Bound (Whisker): {lower_bound}")
        print(f"Upper Bound (Whisker): {upper_bound}")

    def visualize(self):
        # 绘制箱型图
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=self.average_traffic[~np.isnan(self.average_traffic)].flatten())
        plt.title("Boxplot of Average Traffic Flow (Filtered)")
        plt.xlabel("Average Traffic Flow")
        plt.show()
        
        # 创建新的灰度图
        visual_grid = np.full_like(self.average_traffic, 255)
        for row, col in np.vstack((self.zero_indices, self.filtered_indices)):
            visual_grid[row, col] = 0
        
        plt.figure(figsize=(10, 8))
        plt.imshow(visual_grid, cmap='gray')
        plt.title('Filtered Traffic Network Visualization')
        plt.axis('off')
        plt.show()

In [4]:
class TrafficNetworkAnalyzer:
    def __init__(self, zero_indices, filtered_indices, rows=495, cols=436, sigma=2.3):
        self.rows = rows
        self.cols = cols
        self.sigma = sigma
        self.G = nx.DiGraph()  # ✅ 改为有向图
        self.visual_grid = np.full((rows, cols), 255)  # 初始化整个网格
        self.zero_indices = zero_indices
        self.filtered_indices = filtered_indices
        self.flat_node_mapping = {}  # 映射 (row, col) -> flat index

    def to_flat_index(self, r, c):
        """ 将 (row, col) 变换为单索引 """
        return r * self.cols + c

    def construct_graph(self):
        """
        1. 过滤 `zero_indices` 和 `filtered_indices`
        2. 仅添加有效节点，使用 `flat index`
        3. 连接 8 邻居节点（保持方向）
        """
        # 过滤无效节点
        for row, col in self.zero_indices:
            self.visual_grid[row, col] = 0
        for row, col in self.filtered_indices:
            self.visual_grid[row, col] = 0

        # 仅添加有效节点，使用扁平索引
        for r, c in product(range(self.rows), range(self.cols)):
            if self.visual_grid[r, c] == 255:  # 只添加有效节点
                flat_index = self.to_flat_index(r, c)
                self.G.add_node(flat_index)  # 以 flat_index 作为节点
                self.flat_node_mapping[(r, c)] = flat_index  # 记录映射关系

        # 连接 8 邻居（方向为 (r, c) -> (nr, nc)，不会自动加回 (nr, nc) -> (r, c)）
        for (r, c), flat_index in self.flat_node_mapping.items():
            neighbors = [
                (r-1, c), (r+1, c), (r, c-1), (r, c+1),
                (r-1, c-1), (r-1, c+1), (r+1, c-1), (r+1, c+1)
            ]
            for nr, nc in neighbors:
                if (nr, nc) in self.flat_node_mapping:  # 确保是有效点
                    neighbor_index = self.flat_node_mapping[(nr, nc)]
                    self.G.add_edge(flat_index, neighbor_index)  # ✅ 只添加单向边

        # 移除孤立节点
        isolated_nodes = list(nx.isolates(self.G))
        self.G.remove_nodes_from(isolated_nodes)

        print(f"✅ Directed graph constructed: {len(self.G.nodes())} nodes, {len(self.G.edges())} edges.")

    def compute_five_step_neighbors(self):
        """
        计算五阶邻居，并以 (source, target) 形式添加有向边（方向保持原样）
        """
        path_lengths = {node: nx.single_source_shortest_path_length(self.G, node, cutoff=5) for node in self.G.nodes()}

        # 添加五阶内邻接边
        for source, targets in path_lengths.items():
            for target, length in targets.items():
                if length > 0 and length <= 5:
                    self.G.add_edge(source, target, distance=length)  # ✅ 方向保持原样

        print(f"✅ Directed graph expanded to five-step neighbors: {len(self.G.edges())} edges.")

    def apply_gaussian_weights(self):
        """
        使用高斯核计算边权重，并为每个点添加自环（自环权重设为 1）
        然后对每个节点的出边执行 Softmax 归一化，生成 softmax_static_weight，删除原始权重。
        """
        # 计算高斯核权重
        for u, v, data in self.G.edges(data=True):
            d = data['distance']  # 获取路径长度
            weight = np.exp(- (d ** 2) / (2 * self.sigma ** 2))  # 计算高斯核
            self.G[u][v]['weight'] = weight  # 先存入 weight 字段

        # 添加自环（self-loop），权重设为 1
        for node in self.G.nodes():
            self.G.add_edge(node, node, weight=1.0)

        print(f"✅ Gaussian weights applied. Self-loops added.")

        # === Softmax 归一化 ===
        for node in self.G.nodes():
            # 获取该节点的所有出边及其权重
            edges = list(self.G.out_edges(node, data=True))  # 形如 (u, v, data)
            weights = np.array([data['weight'] for _, _, data in edges])  # 提取所有出边的权重

            # 执行 Softmax 归一化
            exp_weights = np.exp(weights)  # 避免溢出
            softmax_weights = exp_weights / np.sum(exp_weights)

            # 更新新的 softmax_static_weight，并删除原始 weight
            for i, (u, v, _) in enumerate(edges):
                self.G[u][v]['softmax_static_weight'] = softmax_weights[i]  # 存入 softmax_static_weight
                del self.G[u][v]['weight']  # 删除原始的高斯核权重

        print(f"✅ Softmax normalization applied to all edges. Original weights removed.")

    def get_graph(self):
        """ 返回最终的 NetworkX 有向图（已扁平化） """
        return self.G

In [5]:
file_path = "/root/autodl-tmp/data/ANTWERP_static.h5"
analyzer = TrafficGridAnalyzer(file_path)
analyzer.load_data()
_, zero_indices = analyzer.analyze_traffic()

march_half_filepath = r"/root/autodl-tmp/data/Antwerp_march_last_half_month_samples_30min_timesteps.txt"
march_filepath = r"/root/autodl-tmp/data/Antwerp_march_last_half_month_samples_30min_timesteps.txt"
    
analyzer = TrafficDataAnalyzer(march_half_filepath, march_filepath)
analyzer.analyze_traffic(zero_indices)
filtered_indices = analyzer.filtered_indices

analyzer = TrafficNetworkAnalyzer(zero_indices, filtered_indices, sigma=2.3)
analyzer.construct_graph()  # 构建图
analyzer.compute_five_step_neighbors()  # 计算五阶邻居
analyzer.apply_gaussian_weights()  # 应用高斯核
G_final = analyzer.get_graph()  # 获取最终图

print(f"✅ Final graph has {len(G_final.nodes())} nodes and {len(G_final.edges())} edges.")

Loaded data shape: (720, 495, 436)
Loaded data shape: (720, 495, 436)
Filtered Traffic Indices (Outside 1.761111111111111-104.5 Range):
      Row  Column
0       0      36
1       0      37
2       0     234
3       0     350
4       0     351
...   ...     ...
2552  493     190
2553  493     263
2554  493     282
2555  494     282
2556  494     414

[2557 rows x 2 columns]
Total number of filtered traffic nodes: 2557
30% Percentile (Lower Bound): 1.761111111111111
95% Percentile (Upper Bound): 104.5
Minimum Value: 1.761111111111111
Q1 (25th Percentile): 3.032638888888889
Median (50th Percentile): 4.995138888888889
Q3 (75th Percentile): 8.754861111111111
Maximum Value: 104.5
Mean: 7.214100918796266
Lower Bound (Whisker): -5.550694444444445
Upper Bound (Whisker): 17.338194444444447
✅ Directed graph constructed: 5524 nodes, 16676 edges.
✅ Directed graph expanded to five-step neighbors: 84768 edges.
✅ Gaussian weights applied. Self-loops added.
✅ Softmax normalization applied to all edges

In [6]:
# ✅ 查询 G_final 中的 node 68 的所有边的高斯核权重
node_id = 68  # 你要查询的节点 ID
edges_68_gaussian = list(G_final.edges(node_id, data=True))  # 获取 node 68 的所有边

# ✅ 打印 node 68 的所有边的 `weight`
print(f"Node {node_id} has {len(edges_68_gaussian)} edges in G_final:")
for u, v, data in edges_68_gaussian:
    print(f"Edge ({u}, {v}) -> Gaussian Weight: {data['softmax_static_weight']:.6f}")

Node 68 has 8 edges in G_final:
Edge (68, 505) -> Gaussian Weight: 0.170344
Edge (68, 941) -> Gaussian Weight: 0.136073
Edge (68, 942) -> Gaussian Weight: 0.136073
Edge (68, 1378) -> Gaussian Weight: 0.105124
Edge (68, 1379) -> Gaussian Weight: 0.105124
Edge (68, 1815) -> Gaussian Weight: 0.085491
Edge (68, 2252) -> Gaussian Weight: 0.075350
Edge (68, 68) -> Gaussian Weight: 0.186421


In [7]:
# ✅ 查询 G_final 中的 node 68 的所有边的高斯核权重
node_id = 505  # 你要查询的节点 ID
edges_68_gaussian = list(G_final.edges(node_id, data=True))  # 获取 node 68 的所有边

# ✅ 打印 node 68 的所有边的 `weight`
print(f"Node {node_id} has {len(edges_68_gaussian)} edges in G_final:")
for u, v, data in edges_68_gaussian:
    print(f"Edge ({u}, {v}) -> Gaussian Weight: {data['softmax_static_weight']:.6f}")

Node 505 has 9 edges in G_final:
Edge (505, 941) -> Gaussian Weight: 0.137867
Edge (505, 68) -> Gaussian Weight: 0.137867
Edge (505, 942) -> Gaussian Weight: 0.137867
Edge (505, 1378) -> Gaussian Weight: 0.110130
Edge (505, 1379) -> Gaussian Weight: 0.110130
Edge (505, 1815) -> Gaussian Weight: 0.085082
Edge (505, 2252) -> Gaussian Weight: 0.069192
Edge (505, 2689) -> Gaussian Weight: 0.060985
Edge (505, 505) -> Gaussian Weight: 0.150879


In [8]:
def merge_gaussian_dtw_weights(G_gaussian, dtw_graphs):
    """
    合并高斯核权重和归一化 DTW 权重，并删除 `normalized_weight`，只保留 `final_weight`。
    
    参数：
        G_gaussian (networkx.Graph): 高斯核加权图（静态）
        dtw_graphs (list of networkx.Graph): 709 张滑动窗口的 DTW 归一化加权图

    返回：
        merged_graphs (list of networkx.Graph): 仅含 `final_weight` 的 709 张合并后的图
    """
    merged_graphs = []
    total_time = 0  # 计算总时间
    
    for i, G_dtw in enumerate(dtw_graphs):
        start_time = time.time()  # 记录开始时间

        G_merged = G_dtw.copy()  # 复制 DTW 图
        
        for u, v in G_dtw.edges():
            gaussian_weight = G_gaussian[u][v]['softmax_static_weight']  # 高斯核权重
            dtw_weight = G_dtw[u][v]['softmax_dynamic_weight']  # 归一化 DTW 权重
            
            # 计算合并后的权重
            merged_weight = 0.5 * gaussian_weight + 0.5 * dtw_weight
            
            # 赋值新的权重
            G_merged[u][v]['merged_weight'] = merged_weight

            # ✅ 删除 `normalized_weight`，减少存储大小
            del G_merged[u][v]['softmax_dynamic_weight']

        merged_graphs.append(G_merged)

        end_time = time.time()  # 记录结束时间
        elapsed_time = end_time - start_time
        total_time += elapsed_time
        
        avg_time_per_graph = total_time / (i + 1)
        remaining_time = avg_time_per_graph * (len(dtw_graphs) - (i + 1))
        print(f"✅ Merged {i+1}/{len(dtw_graphs)} graphs | Time: {elapsed_time:.4f}s | ETA: {remaining_time:.2f}s")

    print(f"✅ Merging complete! Total time: {total_time:.2f}s for {len(merged_graphs)} graphs.")
    return merged_graphs

# ✅ 1. 读取 `DTW` 归一化图
save_dir = r"/root/autodl-tmp/data"
dtw_graphs_path = os.path.join(save_dir, "Softmax_Normalized_DTW_graphs.pkl")

with open(dtw_graphs_path, 'rb') as f:
    dtw_graphs = pickle.load(f)

print(f"✅ Successfully loaded {len(dtw_graphs)} DTW graphs.")

# ✅ 2. 执行权重合并
merged_graphs = merge_gaussian_dtw_weights(G_final, dtw_graphs)

# ✅ 3. 保存合并后的图
merged_graphs_path = os.path.join(save_dir, "Merged_Gaussian_DTW_graphs.pkl")
with open(merged_graphs_path, 'wb') as f:
    pickle.dump(merged_graphs, f)

print(f"✅ Merged graphs saved to '{merged_graphs_path}'.")


✅ Successfully loaded 709 DTW graphs.
✅ Merged 1/709 graphs | Time: 0.3703s | ETA: 262.19s
✅ Merged 2/709 graphs | Time: 0.3473s | ETA: 253.70s
✅ Merged 3/709 graphs | Time: 0.3451s | ETA: 250.11s
✅ Merged 4/709 graphs | Time: 0.3454s | ETA: 248.19s
✅ Merged 5/709 graphs | Time: 0.3472s | ETA: 247.16s
✅ Merged 6/709 graphs | Time: 0.3576s | ETA: 247.58s
✅ Merged 7/709 graphs | Time: 0.3456s | ETA: 246.57s
✅ Merged 8/709 graphs | Time: 0.3471s | ETA: 245.86s
✅ Merged 9/709 graphs | Time: 0.3451s | ETA: 245.07s
✅ Merged 10/709 graphs | Time: 0.3490s | ETA: 244.64s
✅ Merged 11/709 graphs | Time: 0.3443s | ETA: 243.93s
✅ Merged 12/709 graphs | Time: 0.3491s | ETA: 243.56s
✅ Merged 13/709 graphs | Time: 0.3569s | ETA: 243.61s
✅ Merged 14/709 graphs | Time: 0.3457s | ETA: 243.05s
✅ Merged 15/709 graphs | Time: 0.3458s | ETA: 242.52s
✅ Merged 16/709 graphs | Time: 0.3502s | ETA: 242.21s
✅ Merged 17/709 graphs | Time: 0.3451s | ETA: 241.68s
✅ Merged 18/709 graphs | Time: 0.3443s | ETA: 241.14s

In [13]:
# 获取 merged_graphs 中的第 1 张图
G_first = merged_graphs[31]  # 获取第一张合并后的图

# 指定节点 68，查找它的所有边及其权重
node_id = 68  # 你要查询的节点 ID
edges_68 = list(G_first.edges(node_id, data=True))  # 获取 node 68 的所有边

# ✅ 打印 node 68 的所有边的 `final_weight`
print(f"Node {node_id} has {len(edges_68)} edges:")
for u, v, data in edges_68:
    print(f"Edge ({u}, {v}) -> Final Weight: {data['merged_weight']:.6f}")

Node 68 has 8 edges:
Edge (68, 505) -> Final Weight: 0.138672
Edge (68, 941) -> Final Weight: 0.095138
Edge (68, 942) -> Final Weight: 0.122683
Edge (68, 1378) -> Final Weight: 0.098792
Edge (68, 1379) -> Final Weight: 0.087813
Edge (68, 1815) -> Final Weight: 0.128719
Edge (68, 2252) -> Final Weight: 0.096092
Edge (68, 68) -> Final Weight: 0.232092


In [10]:
# ✅ 查询 G_final 中的 node 68 的所有边的高斯核权重
node_id = 68  # 你要查询的节点 ID
edges_68_gaussian = list(G_final.edges(node_id, data=True))  # 获取 node 68 的所有边

# ✅ 打印 node 68 的所有边的 `weight`
print(f"Node {node_id} has {len(edges_68_gaussian)} edges in G_final:")
for u, v, data in edges_68_gaussian:
    print(f"Edge ({u}, {v}) -> Gaussian Weight: {data['softmax_static_weight']:.6f}")

Node 68 has 8 edges in G_final:
Edge (68, 505) -> Gaussian Weight: 0.170344
Edge (68, 941) -> Gaussian Weight: 0.136073
Edge (68, 942) -> Gaussian Weight: 0.136073
Edge (68, 1378) -> Gaussian Weight: 0.105124
Edge (68, 1379) -> Gaussian Weight: 0.105124
Edge (68, 1815) -> Gaussian Weight: 0.085491
Edge (68, 2252) -> Gaussian Weight: 0.075350
Edge (68, 68) -> Gaussian Weight: 0.186421


In [14]:
# 获取 merged_graphs 中的第 1 张图
G_first = dtw_graphs[31]  # 获取第一张合并后的图

# 指定节点 68，查找它的所有边及其权重
node_id = 68  # 你要查询的节点 ID
edges_68 = list(G_first.edges(node_id, data=True))  # 获取 node 68 的所有边

# ✅ 打印 node 68 的所有边的 `final_weight`
print(f"Node {node_id} has {len(edges_68)} edges:")
for u, v, data in edges_68:
    print(f"Edge ({u}, {v}) -> Final Weight: {data['softmax_dynamic_weight']:.6f}")

Node 68 has 8 edges:
Edge (68, 505) -> Final Weight: 0.107000
Edge (68, 941) -> Final Weight: 0.054204
Edge (68, 942) -> Final Weight: 0.109293
Edge (68, 1378) -> Final Weight: 0.092459
Edge (68, 1379) -> Final Weight: 0.070502
Edge (68, 1815) -> Final Weight: 0.171947
Edge (68, 2252) -> Final Weight: 0.116833
Edge (68, 68) -> Final Weight: 0.277763


In [15]:
# 获取 merged_graphs 中的第 1 张图
G_first = merged_graphs[31]  # 获取第一张合并后的图

# 指定节点 68，查找它的所有边及其权重
node_id = 505  # 你要查询的节点 ID
edges_68 = list(G_first.edges(node_id, data=True))  # 获取 node 68 的所有边

# ✅ 打印 node 68 的所有边的 `final_weight`
print(f"Node {node_id} has {len(edges_68)} edges:")
for u, v, data in edges_68:
    print(f"Edge ({u}, {v}) -> Final Weight: {data['merged_weight']:.6f}")


Node 505 has 9 edges:
Edge (505, 941) -> Final Weight: 0.092456
Edge (505, 68) -> Final Weight: 0.125902
Edge (505, 942) -> Final Weight: 0.116430
Edge (505, 1378) -> Final Weight: 0.087735
Edge (505, 1379) -> Final Weight: 0.083574
Edge (505, 1815) -> Final Weight: 0.101043
Edge (505, 2252) -> Final Weight: 0.075369
Edge (505, 2689) -> Final Weight: 0.094169
Edge (505, 505) -> Final Weight: 0.223323
